In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
COEF_DRY_TO_CARBON = 0.40

In [3]:
obs = pd.read_csv("./zoo_2000_2017.csv")
obs.columns = ["month", "zoo"]
obs = obs.set_index("month")
obs.loc[7] = (obs.loc[6] + obs.loc[8]) / 2
obs.loc[12] = (obs.loc[1] + obs.loc[11]) / 2
obs = obs.sort_index()
obs

,zoo
month,
1,1.540
2,0.840
3,1.130
4,1.990
5,6.960
6,8.350
7,9.275
8,10.200
9,8.690


In [4]:
fig = px.line(obs, x=obs.index, y="zoo", title="Zooplankton over Time")
fig.show()

# Climato


In [5]:
time_in_month = pd.date_range(start="2000-01-01", end="2017-12-01", freq="MS")

In [6]:
# Répliquer les données dans obs sur toute la série temporelle time_in_month
obs_replicated = pd.DataFrame(
    np.tile(obs.values, (len(time_in_month) // len(obs) + 1, 1))[: len(time_in_month)],
    index=time_in_month,
    columns=obs.columns,
)
obs_replicated.index.name = "time"

In [7]:
fig = px.line(obs_replicated, x=obs_replicated.index, y="zoo", title="Zooplankton over Time")
fig.show()

In [8]:
obs_replicated

,zoo
time,
2000-01-01,1.54
2000-02-01,0.84
2000-03-01,1.13
2000-04-01,1.99
2000-05-01,6.96
...,...
2017-08-01,10.20
2017-09-01,8.69
2017-10-01,5.59


# Format with forcing


In [9]:
forcing = xr.open_zarr("./Barents_sea_forcing.zarr")
forcing

<xarray.Dataset> Size: 351kB
Dimensions:              (depth: 1, latitude: 1, longitude: 1, time: 7305)
Coordinates:
  * depth                (depth) int64 8B 0
  * latitude             (latitude) int64 8B 72
  * longitude            (longitude) int64 8B 20
  * time                 (time) datetime64[ns] 58kB 1998-01-01 ... 2017-12-31
Data variables:
    T                    (depth, latitude, longitude, time) float64 58kB dask.array<chunksize=(1, 1, 1, 7305), meta=np.ndarray>
    npp                  (latitude, longitude, time) float64 58kB dask.array<chunksize=(1, 1, 7305), meta=np.ndarray>
    pelagic_layer_depth  (latitude, longitude, time) float64 58kB dask.array<chunksize=(1, 1, 7305), meta=np.ndarray>
    zeu                  (latitude, longitude, time) float64 58kB dask.array<chunksize=(1, 1, 7305), meta=np.ndarray>
    zooc                 (latitude, longitude, time) float64 58kB dask.array<chunksize=(1, 1, 7305), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

In [10]:
obs_xr = xr.Dataset.from_dataframe(obs_replicated)
obs_xr = obs_xr.expand_dims({"latitude": forcing.latitude, "longitude": forcing.longitude, "layer": [0]})
obs_xr = obs_xr.transpose("time", "latitude", "longitude", "layer")
obs_xr.time.attrs = forcing.time.attrs
obs_xr.latitude.attrs = forcing.latitude.attrs
obs_xr.longitude.attrs = forcing.longitude.attrs
obs_xr.layer.attrs = {"axis": "Z"}
obs_xr = obs_xr.rename({"zoo": "night"})
obs_xr["night"] = obs_xr.night * COEF_DRY_TO_CARBON
obs_xr.night.attrs = {
    "long_name": "Zooplankton",
    "units": "g / m2",
    "type": "carbon",
    "coef_dry_to_carbon": COEF_DRY_TO_CARBON,
}
obs_xr

<xarray.Dataset> Size: 3kB
Dimensions:    (latitude: 1, longitude: 1, layer: 1, time: 216)
Coordinates:
  * latitude   (latitude) int64 8B 72
  * longitude  (longitude) int64 8B 20
  * layer      (layer) int64 8B 0
  * time       (time) datetime64[ns] 2kB 2000-01-01 2000-02-01 ... 2017-12-01
Data variables:
    night      (time, latitude, longitude, layer) float64 2kB 0.616 ... 0.936

# Export


In [12]:
obs_xr.to_zarr("./zoo_2000_2017.zarr")